In [206]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import codecs
from bs4 import BeautifulSoup
import time
import json
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from multiprocessing.dummy import Value as ThreadValue
import re
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import pymorphy2
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from multiprocessing.dummy import Lock
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from pytorch_transformers import BertTokenizer, BertConfig
from transformers import  BertModel

In [9]:
morph = pymorphy2.MorphAnalyzer()

In [10]:
stemmerR = SnowballStemmer('russian')
stemmerE = SnowballStemmer('english')

In [11]:
def inp_for_titles(line):
    data = line.split('\t', 1)
    doc_id = int(data[0])
    if len(data[1]) == 1:
        title = ''
    else:
        title = data[1]
    doc_to_title[doc_id] = title[:-1]

In [12]:
doc_to_title = {}
with codecs.open('docs_titles.tsv', 'r', 'utf8') as f:
    next(f)
    with ThreadPool(10) as pool:
        pool.map(inp_for_titles, f)
print(len(doc_to_title))

28026


In [13]:
doc_to_title

{15731: 'ВАЗ 21213 | Замена подшипников ступицы | Нива',
 14829: 'Ваз 2107 оптом в Сочи. Сравнить цены, купить потребительские товары на Tiu.ru',
 15764: 'Купить ступица Лада калина2. Трансмиссия - переходные ступицы цена, замена, тюнинг.',
 17669: 'Классика 21010 - 21074',
 14852: 'Ступица Нива — замена подшипника своими руками',
 15458: 'ВАЗ 2110',
 14899: 'Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123',
 16879: 'Купить Подшипники и ступицы FAG (Страница 23)',
 16310: 'HorsePowers — автомобильный интернет портал » Отзыв владельца ВАЗ 2121 Нива 2007 года',
 15440: 'Новости и сообщения из официальной группы Вконтакте торговой компании 33 Sport - Магазины - Тольятти',
 16242: 'Инструкция по замене подшипника передней ступицы ивеко дейли через dorognoekam.ru',
 16383: 'Ступицы - OLX.ua - страница 80',
 15580: 'маааленькая проблемка — бортжурнал Автокам 2160 ╬ 1994 года на DRIVE2',
 16840: 'Разгруженные полуоси для Нива (24 шлица 765 мм)',
 17519: 'Прошивки для нива м7.9.7 скачать - Фа

In [14]:
stop_words = set(stopwords.words(["russian", "english"]))

In [21]:
lock = Lock()

In [23]:
def Parser_for_titles(ID):
    title = (doc_to_title[ID])
    title = title.lower()
    words = [stemmerR.stem(stemmerE.stem(word)) for word in re.sub('[^a-zа-я0-9]', ' ', title).split()]
    with lock:
        vec[ID] = ' '.join([word for word in words])

In [24]:
vec = {}
with ThreadPool(10) as pool:
    pool.map(Parser_for_titles, list(range(1,28027)))

In [25]:
vec

{1: 'м б аншин центр репродукц и генетик фертимед г москв',
 2: 'перевод кив кошельк',
 3: 'проект патрул времен реабилитац духовн существ просмотр тем деньг и экономик обм и финанс деньг без процент',
 4: 'блог клуб преподаван в начальн класс порта профессиональн образован чеченск республик',
 702: 'медицинск порта узбекиста med uz дерматовенеролог рус',
 703: 'уста',
 704: 'как состав бизнес план карьер и сво бизнес',
 705: 'вариац на тем подкормк томат стр 1 томат в теплиц форум',
 706: 'зубн врач и врач стоматолог в чем разниц особен професс',
 1403: 'где в казан можн куп медилис ципер наш справк в город казан татарста',
 1404: 'первичн бесплод причин диагностик профилактик',
 1405: 'стен вконтакт',
 1406: 'экстернат больш не будет',
 1407: 'мыш 2 штук рабоч оптическ цен за 2 мыш куп в красноярск состоян б у клавиатур и мыш на интернет аукцион au ru',
 707: 'как уменьш холестерин в кров в домашн услов народн средств здоров питан и фитнес golbi',
 708: 'современ сатирическ рома лите

In [27]:
with open('titles_stemming.json', 'w', encoding='utf8') as file:
    json.dump(vec, file)

Застемил заголовки - уже прекрасно!

## Мутим фичи

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-macosx_10_12_x86_64.whl size=324287 sha256=1aca536a5f62e01a9433b11746a610136aa323a703f84c1f45af47de11095c96
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 2.2 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108249 sha256=54b93ee534545cb3657fe4c27116c6ea53bd50ab3c37028568418f11c50e2d07
  Stored in directory: /Users/egor/Library/Caches/pip/wheels/34/3d/14/f19c01a19c9201cdb6a76b049904d5226912569be919ad1eae
Successfully built smart-open
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [14]:
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.fasttext import load_facebook_model

Взяли заголовки

In [15]:
vec = {}

In [16]:
with open('titles_stemming.json', 'r', encoding='utf8') as file:
    vec = json.load(file)

In [17]:
sentences = []
for value in vec.values():
    sentences.append(value)

In [23]:
model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')

In [24]:
len(sentences)

28026

In [25]:
first = sentences[0]
first

'м б аншин центр репродукц и генетик фертимед г москв'

In [27]:
len(model_fasttext.wv[first])

100

In [230]:
class Import:
    def __init__(self):
        self.doc_to_title = {}
        #self.model_fasttext = FastText('/Users/egor/Desktop/Техносфера/MachineLearning/project/ft_freqprune_400K_100K_pq_300.bin')
        with open('/Users/egor/Desktop/Техносфера/MachineLearning/project/docs_titles.tsv') as f:
            for num_line, line in enumerate(f):
                if num_line == 0:
                    continue
                data = line.strip().split('\t', 1)
                doc_id = int(data[0])
                if len(data) == 1:
                    title = ''
                else:
                    title = data[1]
                self.doc_to_title[doc_id] = title
    
    def make_train(self):
        train_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/train_groups.csv')
        traingroups_titledata = {}
        for i in range(len(train_data)):
            new_doc = train_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            target = new_doc['target']
            title = self.doc_to_title[doc_id]
            if doc_group not in traingroups_titledata:
                traingroups_titledata[doc_group] = []
            traingroups_titledata[doc_group].append((doc_id, title, target))
        
        y_train = []
        X_train = []
        groups_train = []
        for new_group in traingroups_titledata:
            docs = traingroups_titledata[new_group]
            for k, (doc_id, title, target_id) in enumerate(docs):
                y_train.append(target_id)
                groups_train.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j, target_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_train.append(sorted(all_dist, reverse=True)[0:25]    )
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        groups_train = np.array(groups_train)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.empty((train_data.shape[0], 3)) # кол-во новых фичей
        groups = train_data.groupby('group_id')
        for group_idx, titles_id in groups.groups.items():
            embedded_titles = model_fasttext.wv[titles_id.values]
            model = DBSCAN(0.45, metric="cosine", min_samples=5)
            labels = model.fit_predict(embedded_titles)
            for l in range(len(labels)):
                if labels[l] == -1:
                    My_X[i + l, 0] = 0
                else:
                    My_X[i + l, 0] = 1
                min_d = 10
                max_d = 0
                for k in range(len(labels)):
                    if k != l:
                        d = cosine(embedded_titles[l], embedded_titles[k])
                        if min_d > d:
                            min_d = d
                        if d > max_d:
                            max_d = d
                My_X[i + l, 1] = min_d
                My_X[i + l, 2] = max_d
            i = len(titles_id)
        
        
        # хочу посчитать косинусное расстояние между всеми объектами и найти максимальное и минимальное
        X_train = np.concatenate((X_train, My_X), axis=1)
        return X_train, y_train, groups_train
    
    def make_test(self):
        test_data = pd.read_csv('/Users/egor/Desktop/Техносфера/MachineLearning/project/test_groups.csv')
        testgroups_titledata = {}
        for i in range(len(test_data)):
            new_doc = test_data.iloc[i]
            doc_group = new_doc['group_id']
            doc_id = new_doc['doc_id']
            title = self.doc_to_title[doc_id]
            if doc_group not in testgroups_titledata:
                testgroups_titledata[doc_group] = []
            testgroups_titledata[doc_group].append((doc_id, title))
            
        X_test = []
        groups_test = []
        for new_group in testgroups_titledata:
            docs = testgroups_titledata[new_group]
            for k, (doc_id, title) in enumerate(docs):
                groups_test.append(new_group)
                all_dist = []
                words = set(title.strip().split())
                for j in range(0, len(docs)):
                    if k == j:
                        continue
                    doc_id_j, title_j = docs[j]
                    words_j = set(title_j.strip().split())
                    all_dist.append(len(words.intersection(words_j)))
                X_test.append(sorted(all_dist, reverse=True)[0:25])
        X_test = np.array(X_test)
        
        # Класстеризация внутри группы:
        i = 0
        My_X = np.empty((test_data.shape[0], 3))
        groups = test_data.groupby('group_id')
        for group_idx, titles_id in groups.groups.items():
            embedded_titles = model_fasttext.wv[titles_id.values]
            model = DBSCAN(0.45, metric="cosine", min_samples=5)
            labels = model.fit_predict(embedded_titles)
            for l in range(len(labels)):
                if labels[l] == -1:
                    My_X[i + l, 0] = 0
                else:
                    My_X[i + l, 0] = 1
                min_d = 10
                max_d = 0
                for k in range(len(labels)):
                    if k != l:
                        d = cosine(embedded_titles[l], embedded_titles[k])
                        if min_d > d:
                            min_d = d
                        if d > max_d:
                            max_d = d
                My_X[i + l, 1] = min_d
                My_X[i + l, 2] = max_d
            i = len(titles_id)
        
        X_test = np.concatenate((X_test, My_X), axis=1)
        groups_test = np.array(groups_test)
        return X_test, groups_test, test_data['pair_id'].tolist()

In [231]:
importer = Import()

In [232]:
X_train, y_train, groups_train = importer.make_train()

In [233]:
X_test, groups_test, pair_ids = importer.make_test()

## Модельки

In [175]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from statistics import mean
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neural_network import MLPClassifier
import json
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import catboost
from sklearn.svm import SVC

In [241]:
my_sk = StandardScaler()
X_train_sk = my_sk.fit_transform(X_train)
X_test_sk = my_sk.fit_transform(X_test)

In [261]:
clf_e_1 = RandomForestClassifier(max_depth=10)
clf_e_2 = ExtraTreesClassifier(max_depth=15)
clf_e_3 = LogisticRegression()
clf_e_4 = catboost.CatBoostClassifier()
clf_e_5 = MLPClassifier(max_iter=800, hidden_layer_sizes=(10, 10), random_state=42)
clf_e_6 = KNeighborsClassifier(10)

In [262]:
clf_e_1.fit(X_train_sk, y_train)
clf_e_2.fit(X_train_sk, y_train)
clf_e_3.fit(X_train_sk, y_train)
clf_e_4.fit(X_train_sk, y_train)
clf_e_5.fit(X_train_sk, y_train)
clf_e_6.fit(X_train_sk, y_train)

Learning rate set to 0.029437
0:	learn: 0.6783540	total: 13.9ms	remaining: 13.9s
1:	learn: 0.6646499	total: 31.3ms	remaining: 15.6s
2:	learn: 0.6515154	total: 47.2ms	remaining: 15.7s
3:	learn: 0.6394138	total: 59ms	remaining: 14.7s
4:	learn: 0.6280032	total: 69.4ms	remaining: 13.8s
5:	learn: 0.6178138	total: 79.4ms	remaining: 13.1s
6:	learn: 0.6084744	total: 89.8ms	remaining: 12.7s
7:	learn: 0.5997293	total: 100ms	remaining: 12.4s
8:	learn: 0.5904974	total: 111ms	remaining: 12.2s
9:	learn: 0.5825065	total: 121ms	remaining: 11.9s
10:	learn: 0.5745176	total: 131ms	remaining: 11.8s
11:	learn: 0.5674479	total: 141ms	remaining: 11.6s
12:	learn: 0.5603573	total: 152ms	remaining: 11.5s
13:	learn: 0.5540733	total: 163ms	remaining: 11.5s
14:	learn: 0.5479123	total: 173ms	remaining: 11.3s
15:	learn: 0.5422389	total: 184ms	remaining: 11.3s
16:	learn: 0.5371240	total: 195ms	remaining: 11.3s
17:	learn: 0.5322701	total: 212ms	remaining: 11.6s
18:	learn: 0.5272214	total: 228ms	remaining: 11.8s
19:	le

174:	learn: 0.4418560	total: 2.09s	remaining: 9.85s
175:	learn: 0.4418128	total: 2.1s	remaining: 9.85s
176:	learn: 0.4416887	total: 2.12s	remaining: 9.84s
177:	learn: 0.4416271	total: 2.13s	remaining: 9.82s
178:	learn: 0.4415295	total: 2.14s	remaining: 9.81s
179:	learn: 0.4414704	total: 2.15s	remaining: 9.79s
180:	learn: 0.4414335	total: 2.16s	remaining: 9.78s
181:	learn: 0.4413851	total: 2.18s	remaining: 9.8s
182:	learn: 0.4413492	total: 2.2s	remaining: 9.81s
183:	learn: 0.4413119	total: 2.21s	remaining: 9.82s
184:	learn: 0.4412732	total: 2.23s	remaining: 9.84s
185:	learn: 0.4412292	total: 2.26s	remaining: 9.88s
186:	learn: 0.4411042	total: 2.27s	remaining: 9.88s
187:	learn: 0.4410324	total: 2.29s	remaining: 9.88s
188:	learn: 0.4409791	total: 2.31s	remaining: 9.9s
189:	learn: 0.4409027	total: 2.33s	remaining: 9.93s
190:	learn: 0.4408522	total: 2.35s	remaining: 9.97s
191:	learn: 0.4408215	total: 2.37s	remaining: 9.99s
192:	learn: 0.4407803	total: 2.4s	remaining: 10s
193:	learn: 0.44074

335:	learn: 0.4331511	total: 4.18s	remaining: 8.27s
336:	learn: 0.4331453	total: 4.2s	remaining: 8.26s
337:	learn: 0.4331148	total: 4.21s	remaining: 8.25s
338:	learn: 0.4330531	total: 4.22s	remaining: 8.23s
339:	learn: 0.4330109	total: 4.23s	remaining: 8.22s
340:	learn: 0.4329118	total: 4.24s	remaining: 8.2s
341:	learn: 0.4328379	total: 4.25s	remaining: 8.19s
342:	learn: 0.4327969	total: 4.27s	remaining: 8.17s
343:	learn: 0.4327032	total: 4.28s	remaining: 8.16s
344:	learn: 0.4326216	total: 4.29s	remaining: 8.14s
345:	learn: 0.4325014	total: 4.3s	remaining: 8.13s
346:	learn: 0.4324228	total: 4.31s	remaining: 8.11s
347:	learn: 0.4323675	total: 4.32s	remaining: 8.1s
348:	learn: 0.4323233	total: 4.33s	remaining: 8.08s
349:	learn: 0.4322759	total: 4.34s	remaining: 8.07s
350:	learn: 0.4322493	total: 4.35s	remaining: 8.05s
351:	learn: 0.4321996	total: 4.37s	remaining: 8.04s
352:	learn: 0.4321229	total: 4.38s	remaining: 8.02s
353:	learn: 0.4321004	total: 4.39s	remaining: 8.01s
354:	learn: 0.43

495:	learn: 0.4257681	total: 6.03s	remaining: 6.13s
496:	learn: 0.4257367	total: 6.05s	remaining: 6.12s
497:	learn: 0.4256043	total: 6.07s	remaining: 6.12s
498:	learn: 0.4255677	total: 6.09s	remaining: 6.11s
499:	learn: 0.4255170	total: 6.1s	remaining: 6.1s
500:	learn: 0.4254894	total: 6.11s	remaining: 6.09s
501:	learn: 0.4254227	total: 6.12s	remaining: 6.07s
502:	learn: 0.4253366	total: 6.13s	remaining: 6.06s
503:	learn: 0.4253058	total: 6.14s	remaining: 6.05s
504:	learn: 0.4252395	total: 6.16s	remaining: 6.04s
505:	learn: 0.4251821	total: 6.18s	remaining: 6.03s
506:	learn: 0.4251477	total: 6.19s	remaining: 6.02s
507:	learn: 0.4251286	total: 6.21s	remaining: 6.01s
508:	learn: 0.4251066	total: 6.23s	remaining: 6.01s
509:	learn: 0.4251010	total: 6.24s	remaining: 6s
510:	learn: 0.4250935	total: 6.25s	remaining: 5.98s
511:	learn: 0.4250716	total: 6.26s	remaining: 5.97s
512:	learn: 0.4250298	total: 6.27s	remaining: 5.96s
513:	learn: 0.4249985	total: 6.28s	remaining: 5.94s
514:	learn: 0.424

668:	learn: 0.4190533	total: 8.08s	remaining: 4s
669:	learn: 0.4190106	total: 8.09s	remaining: 3.99s
670:	learn: 0.4189474	total: 8.11s	remaining: 3.97s
671:	learn: 0.4188907	total: 8.12s	remaining: 3.96s
672:	learn: 0.4188182	total: 8.13s	remaining: 3.95s
673:	learn: 0.4187904	total: 8.14s	remaining: 3.94s
674:	learn: 0.4187126	total: 8.15s	remaining: 3.92s
675:	learn: 0.4186645	total: 8.16s	remaining: 3.91s
676:	learn: 0.4186304	total: 8.17s	remaining: 3.9s
677:	learn: 0.4186059	total: 8.18s	remaining: 3.89s
678:	learn: 0.4185727	total: 8.2s	remaining: 3.87s
679:	learn: 0.4185684	total: 8.21s	remaining: 3.86s
680:	learn: 0.4185146	total: 8.22s	remaining: 3.85s
681:	learn: 0.4184832	total: 8.23s	remaining: 3.84s
682:	learn: 0.4184693	total: 8.24s	remaining: 3.82s
683:	learn: 0.4184502	total: 8.25s	remaining: 3.81s
684:	learn: 0.4183773	total: 8.26s	remaining: 3.8s
685:	learn: 0.4183145	total: 8.28s	remaining: 3.79s
686:	learn: 0.4183115	total: 8.29s	remaining: 3.78s
687:	learn: 0.4182

838:	learn: 0.4141506	total: 10.1s	remaining: 1.95s
839:	learn: 0.4141326	total: 10.2s	remaining: 1.93s
840:	learn: 0.4141280	total: 10.2s	remaining: 1.92s
841:	learn: 0.4140566	total: 10.2s	remaining: 1.91s
842:	learn: 0.4140546	total: 10.2s	remaining: 1.9s
843:	learn: 0.4140529	total: 10.2s	remaining: 1.88s
844:	learn: 0.4139935	total: 10.2s	remaining: 1.87s
845:	learn: 0.4139472	total: 10.2s	remaining: 1.86s
846:	learn: 0.4139161	total: 10.2s	remaining: 1.85s
847:	learn: 0.4139152	total: 10.2s	remaining: 1.83s
848:	learn: 0.4138839	total: 10.2s	remaining: 1.82s
849:	learn: 0.4138211	total: 10.3s	remaining: 1.81s
850:	learn: 0.4138049	total: 10.3s	remaining: 1.8s
851:	learn: 0.4137576	total: 10.3s	remaining: 1.78s
852:	learn: 0.4137388	total: 10.3s	remaining: 1.77s
853:	learn: 0.4137182	total: 10.3s	remaining: 1.76s
854:	learn: 0.4137069	total: 10.3s	remaining: 1.75s
855:	learn: 0.4136385	total: 10.3s	remaining: 1.74s
856:	learn: 0.4136274	total: 10.3s	remaining: 1.72s
857:	learn: 0.

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [263]:
res = (clf_e_1.predict_proba(X_train_sk)[:, 1] + clf_e_2.predict_proba(X_train_sk)[: ,1] + clf_e_3.predict_proba(X_train_sk)[:, 1]
+ clf_e_4.predict_proba(X_train_sk)[:, 1] + clf_e_5.predict_proba(X_train_sk)[:, 1] + clf_e_6.predict_proba(X_train_sk)[:, 1])/ 6

In [264]:
for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
    print(f1_score(y_train, np.array(res > th, dtype=int)))

0.5467451523545707
0.6443755027002183
0.6825717751718561
0.6902491541064288
0.6684064022268615
0.5822387767104489
0.5170876671619614


In [267]:
f1_score(y_train, np.array(res > 0.35, dtype=int))

0.6939245608983586

In [268]:
test = (clf_e_1.predict_proba(X_test_sk)[:, 1] + clf_e_2.predict_proba(X_test_sk)[: ,1] + clf_e_3.predict_proba(X_test_sk)[:, 1]
+ clf_e_4.predict_proba(X_test_sk)[:, 1] + clf_e_5.predict_proba(X_test_sk)[:, 1] + clf_e_6.predict_proba(X_test_sk)[:, 1])/ 6

In [269]:
y_test = np.array(test > 0.35, dtype=int)

In [270]:
y_test.shape

(16627,)

In [271]:
with open("/Users/egor/Desktop/resultBest.csv", "w", encoding='utf-8') as outfile:
    outfile.write(str("pair_id,target\n"))
    for i in range(len(y_test)):
        outfile.write(str(pair_ids[i]) + str(",") + str(y_test[i]) + str("\n"))